# Step1: 環境設置

In [1]:
!git clone https://github.com/ma-compbio/Higashi/

Cloning into 'Higashi'...
remote: Enumerating objects: 1268, done.
remote: Total 1268 (delta 0), reused 0 (delta 0), pack-reused 1268
Receiving objects: 100% (1268/1268), 87.04 MiB | 4.13 MiB/s, done.
Resolving deltas: 100% (781/781), done.


In [2]:
%cd Higashi

/content/Higashi


In [3]:
%ll

total 28
drwxr-xr-x 2 root 4096 Jun 15 15:03 conda/
drwxr-xr-x 2 root 4096 Jun 15 15:03 figs/
drwxr-xr-x 5 root 4096 Jun 15 15:03 higashi/
-rw-r--r-- 1 root 1070 Jun 15 15:03 LICENSE
-rw-r--r-- 1 root 2653 Jun 15 15:03 README.md
-rw-r--r-- 1 root  796 Jun 15 15:03 setup.py
drwxr-xr-x 2 root 4096 Jun 15 15:03 tutorials/


In [4]:
import torch
print(torch.cuda.is_available())

True


In [5]:
!nvidia-smi

Wed Jun 15 15:03:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 安裝

#### 使用 pip + requirements.txt

In [6]:
%%shell

cat >> ./requirements.txt << EOF
numpy>=1.21.2
scipy==1.7.3
pandas==1.3.4
cython>=0.29.24
torch>=1.8.0
fbpca
scikit-learn>=0.23.2
tqdm
h5py
cooler>=0.8
seaborn>=0.11.2
umap-learn>=0.5
bokeh>=2.1.1
Pillow
EOF

In [7]:
!pip3 install -r "./requirements.txt"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 11.3 MB 57.9 MB/s 
     |████████████████████████████████| 100 kB 12.7 MB/s 
     |████████████████████████████████| 88 kB 8.4 MB/s 
     |████████████████████████████████| 130 kB 78.6 MB/s 
     |████████████████████████████████| 102 kB 65.5 MB/s 
     |████████████████████████████████| 65 kB 5.1 MB/s 
     |████████████████████████████████| 475 kB 66.1 MB/s 
     |████████████████████████████████| 1.1 MB 50.0 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=65788c36c8f30a5e3029f660aa6910330a33880a9a04f9fc6a072b198fd9e585
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for cytoolz: filename=cytoolz-0.10.1-cp37-cp37m-linux_x86_64.whl size=1238259 sha256=ca8c9043d4291abd

#### 使用 setup.py (unuse)

In [8]:
# !sudo python setup.py install # 沒有用

# Step 2: prpare data: Ramani

In [9]:
!pwd

/content/Higashi


In [10]:
import gdown
id = "1NX6MGSz3JJ_7iN8VDZjjpMlLaqlpA60t" 
# id = "1NWFwjaYT9RxqDUK87h5c8SVoO_WZbbWV" # 4DN
gdown.download(id=id, output="Ramani et al.zip", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NX6MGSz3JJ_7iN8VDZjjpMlLaqlpA60t
To: /content/Higashi/Ramani et al.zip
100%|██████████| 256M/256M [00:00<00:00, 322MB/s]


'Ramani et al.zip'

In [11]:
!mkdir Ramani

In [12]:
!pwd

/content/Higashi


In [13]:
!unzip 'Ramani et al.zip' -d Ramani

Archive:  Ramani et al.zip
   creating: Ramani/Ramani et al/
  inflating: Ramani/Ramani et al/config_ramani.JSON  
  inflating: Ramani/Ramani et al/cytoBand_hg19.txt  
  inflating: Ramani/Ramani et al/data.txt  
  inflating: Ramani/Ramani et al/hg19.chrom.sizes.txt  
  inflating: Ramani/Ramani et al/label_info.pickle  


In [14]:
!mv "Ramani/Ramani et al/config_ramani.JSON" "Ramani/config_ramani.JSON"
!mv "Ramani/Ramani et al/data.txt" "Ramani/data.txt"
!mv "Ramani/Ramani et al/label_info.pickle" "Ramani/label_info.pickle"

## genome reference
Path of the genome reference file from USCS Genome Browser, will be used to generate bin nodes
- 下載: https://www.genome.ucsc.edu/goldenpath/help/hg19.chrom.sizes

In [15]:
!wget https://www.genome.ucsc.edu/goldenpath/help/hg19.chrom.sizes

--2022-06-15 15:04:45--  https://www.genome.ucsc.edu/goldenpath/help/hg19.chrom.sizes
Resolving www.genome.ucsc.edu (www.genome.ucsc.edu)... 128.114.119.131, 128.114.119.132
Connecting to www.genome.ucsc.edu (www.genome.ucsc.edu)|128.114.119.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1971 (1.9K)
Saving to: ‘hg19.chrom.sizes’

hg19.chrom.sizes    100%[===================>]   1.92K  --.-KB/s    in 0s      

2022-06-15 15:04:46 (108 MB/s) - ‘hg19.chrom.sizes’ saved [1971/1971]



In [16]:
!mv ./hg19.chrom.sizes ./Ramani/

## cytoband reference
Path of the cytoband reference file from USCS Genome Browser, will be used to remove centromere regions

In [17]:
!wget https://raw.githubusercontent.com/waldronlab/subtypeHeterogeneity/master/inst/extdata/cytoBand_hg19.txt

--2022-06-15 15:04:46--  https://raw.githubusercontent.com/waldronlab/subtypeHeterogeneity/master/inst/extdata/cytoBand_hg19.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30754 (30K) [text/plain]
Saving to: ‘cytoBand_hg19.txt’

cytoBand_hg19.txt   100%[===================>]  30.03K  --.-KB/s    in 0.003s  

2022-06-15 15:04:47 (9.72 MB/s) - ‘cytoBand_hg19.txt’ saved [30754/30754]



In [18]:
!mv cytoBand_hg19.txt ./Ramani/cytoBand_hg19.txt

In [19]:
%ll ./Ramani

total 884484
-rw-r--r-- 1 root      1318 Jan 25 23:22  config_ramani.JSON
-rw-r--r-- 1 root     30754 Jun 15 15:04  cytoBand_hg19.txt
-rw-r--r-- 1 root 905651666 Jan  4 12:03  data.txt
-rw-r--r-- 1 root      1971 Jun 22  2009  hg19.chrom.sizes
-rw-r--r-- 1 root      8048 Jan  4 11:59  label_info.pickle
drwxr-xr-x 2 root      4096 Jun 15 15:04 'Ramani et al'/


### 看一下 config_ramani.JSON"

In [20]:
!cat "Ramani/config_ramani.JSON"

{
  "config_name": "ramani et. al",
  "data_dir": "/work/magroup/ruochiz/Data/scHiC_collection/ramani",
  "temp_dir": "/work/magroup/ruochiz/Higashi/Temp/ramani",
  "genome_reference_path": "/work/magroup/ruochiz/Higashi/hg19.chrom.sizes.txt",
  "cytoband_path": "/work/magroup/ruochiz/Higashi/cytoBand_hg19.txt",
  "chrom_list": ["chr1","chr2","chr3","chr4","chr5",
			  "chr6","chr7","chr8","chr9","chr10",
			  "chr11","chr12","chr13","chr14","chr15",
			  "chr16","chr17","chr18","chr19","chr20",
			  "chr21","chr22","chrX"],
  "resolution": 1000000,
  "resolution_cell": 1000000,
  "minimum_distance": 2000000,
  "maximum_distance": -1,
  "local_transfer_range": 1,
  "dimensions": 64,
  "impute_list":["chr1","chr2","chr3","chr4","chr5",
			  "chr6","chr7","chr8","chr9","chr10",
			  "chr11","chr12","chr13","chr14","chr15",
			  "chr16","chr17","chr18","chr19","chr20",
			  "chr21","chr22","chrX"],
  "minimum_impute_distance": 0,
  "maximum_impute_distance": -1,
  "neighbor_num": 5,
  "pl

## Step 3: 改參數 
./Ramani/my_config_ramani.JSON

In [21]:
%%shell

cat >> ./Ramani/my_config_ramani.JSON << EOF
{
  "config_name": "ramani et. al",
  "data_dir": "Ramani",
  "temp_dir": "Ramani/temp",
  "genome_reference_path": "Ramani/hg19.chrom.sizes",
  "cytoband_path": "Ramani/cytoBand_hg19.txt",
  "chrom_list": ["chr1","chr2","chr3","chr4","chr5",
			  "chr6","chr7","chr8","chr9","chr10",
			  "chr11","chr12","chr13","chr14","chr15",
			  "chr16","chr17","chr18","chr19","chr20",
			  "chr21","chr22","chrX"],
  "resolution": 1000000,
  "resolution_cell": 1000000,
  "minimum_distance": 2000000,
  "maximum_distance": -1,
  "local_transfer_range": 1,
  "dimensions": 64,
  "impute_list":["chr1","chr2","chr3","chr4","chr5",
			  "chr6","chr7","chr8","chr9","chr10",
			  "chr11","chr12","chr13","chr14","chr15",
			  "chr16","chr17","chr18","chr19","chr20",
			  "chr21","chr22","chrX"],
  "minimum_impute_distance": 0,
  "maximum_impute_distance": -1,
  "neighbor_num": 5,
  "plot_start": 0,
  "plot_end": -1,
  "plot_label": ["cell type"],
  "call_tads": false,
  "embedding_name": "exp_zinb3",
  "cpu_num": -1,
  "gpu_num": 1,
  "optional_smooth": false,
  "optional_quantile": false,
  "rank_thres": 1,
  "loss_mode": "zinb",
  "random_walk": false,
    "UMAP_params": {"n_neighbors": 20
  }
}
EOF

In [22]:

import gdown
id = "1GBpZwRW8ccrS1EFWwZ8QTgjxv-tSvhLD"
gdown.download(id=id, output="runincolab.py", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1GBpZwRW8ccrS1EFWwZ8QTgjxv-tSvhLD
To: /content/Higashi/runincolab.py
100%|██████████| 2.58k/2.58k [00:00<00:00, 3.12MB/s]


'runincolab.py'

In [23]:
!pwd

/content/Higashi


## Step 5: 訓練 Higashi

In [ ]:
!python runincolab.py

generating start/end dict for chromosome
extracting from data.txt
100% 15891786/15891786 [00:49<00:00, 321713.97it/s]
generating contact maps for baseline
data loaded
4110311 False
creating matrices tasks: 100% 23/23 [00:21<00:00,  1.07it/s]
total_feats_size 403
100% 23/23 [00:04<00:00,  4.69it/s]
setting to gpu:0
training on data from: ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX']
total_sparsity_cell 0.025731658257764963
no contractive loss
batch_size 256
Node type num [620 250 244 199 192 181 172 160 147 142 136 136 134 116 108 103  91  82
  79  60  64  49  52 156] [ 620  870 1114 1313 1505 1686 1858 2018 2165 2307 2443 2579 2713 2829
 2937 3040 3131 3213 3292 3352 3416 3465 3517 3673]
start making attribute
0.534:  47% 141/300 [00:00<00:00, 419.76it/s]
loss 0.5341129899024963 loss best 0.5176699161529541 epochs 141

initializing data